In [1]:
# # !pip install pymorphy2
# !wget -O pymorphy2-dicts-ru.tar.gz https://files.pythonhosted.org/packages/b2/b4/732ff6eeac8c9ea22e7e1c7a321b21b6f3ba19d5e0a8925f35da9c8ebbb2/pymorphy2-dicts-ru-2.4.404381.4453942.tar.gz

In [2]:
# !pip install fasttext-wheel

In [3]:
import pickle
import pandas as pd
import tarfile
import gc
import ast
import re

from tqdm import tqdm
# from multiprocessing import Pool

from string import punctuation

import fasttext
import pymorphy2

# from nltk.tokenize import word_tokenize
from razdel import tokenize
from nltk.corpus import stopwords

In [4]:
TRAIN_MODEL = True
PREPROCESS_DATA = not TRAIN_MODEL
DESC_N = 1000

In [14]:
import nltk
nltk.download('stopwords')
stop = stopwords.words('russian')
punkt = [p for p in punctuation] + ["`", "``" , "''", "'"]

# rudict = tarfile.open('pymorphy2-dicts-ru.tar.gz')
# rudict.extractall()
# rudict.close()

# lemmatizer = pymorphy2.MorphAnalyzer(
#     path='./pymorphy2-dicts-ru-2.4.404381.4453942/pymorphy2_dicts_ru/data', 
#     lang='ru'
# )

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# !pip install pymorphy3
# !pip install -U pymorphy2-dicts-ru

In [7]:
import pymorphy3 as pm
morph = pm.MorphAnalyzer(lang='ru')

In [8]:
morph.normal_forms('Винтажная люстра')

['винтажная люстр', 'винтажная люстрый', 'винтажная люстра']

In [28]:
train_df = pd.read_parquet('train_zalupa.parquet')

In [29]:
train_df

,variantid_1,variantid_2,group_id,name_1,name_2,description_1,description_2,category_level_1_1,category_level_1_2,category_level_2_1,category_level_2_2,category_level_3_1,category_level_3_2,category_level_4_1,category_level_4_2,price_1,price_2,characteristic_attributes_mapping_1,characteristic_attributes_mapping_2,is_double
0,0000102c4b265b3c6346920adc5970d50d4a9ec1ad2709...,1a395516c4f953404b82c568803bc98c880053731cbcfa...,134041,Угловая полка доя икон,Подсвечник давленный 5,35 x35x50. Забрать можно сегодня до 12.00,Подсвечник давленный номер 5 высота 22-23 см ...,Для дома и дачи,Для дома и дачи,Мебель и интерьер,Мебель и интерьер,Другое,"Предметы интерьера, искусство",None,Ароматы для дома и свечи,1000.0,8000.0,"{""45"": ""155"", ""2832"": ""20047"", ""110064"": ""1022...","{""45"": ""146"", ""2832"": ""20047"", ""110064"": ""5650...",0
1,0000102c4b265b3c6346920adc5970d50d4a9ec1ad2709...,881041c5342b13b2bb88e5081e5ac6f98f633b0ac03345...,134041,Угловая полка доя икон,Тумба под телевизор,35 x35x50. Забрать можно сегодня до 12.00,Тумба в хорошем состоянии,Для дома и дачи,Для дома и дачи,Мебель и интерьер,Мебель и интерьер,Другое,Подставки и тумбы,None,Тумбы,1000.0,3000.0,"{""45"": ""155"", ""2832"": ""20047"", ""110064"": ""1022...","{""45"": ""151"", ""2832"": ""20047"", ""110064"": ""4800...",0
2,0000102c4b265b3c6346920adc5970d50d4a9ec1ad2709...,e3cf5f4187dc4762ac8272e627f431b137e12038313149...,134041,Угловая полка доя икон,Свеча,35 x35x50. Забрать можно сегодня до 12.00,Размер упаковки 13x18x11 . Цена за одну свечу...,Для дома и дачи,Для дома и дачи,Мебель и интерьер,Мебель и интерьер,Другое,"Предметы интерьера, искусство",None,Ароматы для дома и свечи,1000.0,1000.0,"{""45"": ""155"", ""2832"": ""20047"", ""110064"": ""1022...","{""45"": ""146"", ""2832"": ""20047"", ""110064"": ""5650...",0
3,000034234fb64b71e42f8b2578382ef9904c96ade6f651...,2f684031148849040d1e490109486157791bdf46ec0439...,86141,Наушники Apple AirPods Max Lux,Камера видеонаблюдения WiFi Ezviz C6CN,САМОВЫВО3А HΕТ! ТОЛЬКО АВИТО ДОСТАВКА! ПΕРΕД О...,САМОВЫВО3А HΕТ! ТОЛЬКО АВИТО ДОСТАВКА! ПΕРΕД О...,Электроника,Электроника,Аудио и видео,Аудио и видео,Наушники,Видеокамеры,None,None,2000.0,4200.0,"{""132"": ""5034"", ""2816"": ""20007"", ""110064"": ""49...","{""132"": ""605"", ""2816"": ""20007"", ""110064"": ""494...",0
4,000034234fb64b71e42f8b2578382ef9904c96ade6f651...,3086b118443b3c97125fdb1baa97a24eac5a19a1105327...,86141,Наушники Apple AirPods Max Lux,Камера видеонаблюдения уличная WiFi Imou IPC-S...,САМОВЫВО3А HΕТ! ТОЛЬКО АВИТО ДОСТАВКА! ПΕРΕД О...,ОТПРАВКА ТОЛЬКО ЧEРE3 АВИТО ДОСТАВКУ ‼️\nСАМОВ...,Электроника,Электроника,Аудио и видео,Аудио и видео,Наушники,Видеокамеры,None,None,2000.0,9500.0,"{""132"": ""5034"", ""2816"": ""20007"", ""110064"": ""49...","{""132"": ""605"", ""2816"": ""20007"", ""110064"": ""494...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1879550,ffffec0a542272cfff179d041461cdc2fef075a7b225ec...,6061ea0c2fe44e45699fab71273a2edb5bc87a33713655...,20878,"Xiaomi Redmi Note 12S, 8/256 ГБ","realme 12 Pro+, 12/512 ГБ",Xiaomi Redmi Note 12S NFC 256Гб \ 8Гб (+3Гб RA...,Смартфон Realme 12 Pro+ 5G 512Гб \ 12Гб (+3Гб)...,Электроника,Электроника,Телефоны,Телефоны,Мобильные телефоны,Мобильные телефоны,None,None,15990.0,38990.0,"{""143"": ""18691"", ""2822"": ""20018"", ""110064"": ""1...","{""143"": ""632"", ""2822"": ""20018"", ""110064"": ""111...",0
1879551,ffffec0a542272cfff179d041461cdc2fef075a7b225ec...,f5511cfb17b50b3fa02fd2a77836b2e202391cfcbacd02...,20878,"Xiaomi Redmi Note 12S, 8/256 ГБ","Xiaomi Redmi Note 13 Pro+, 12/512 ГБ",Xiaomi Redmi Note 12S NFC 256Гб \ 8Гб (+3Гб RA...,Xiaomi Redmi Note 13 Pro+ 5G 512Гб \ 12Гб \Mid...,Электроника,Электроника,Телефоны,Телефоны,Мобильные телефоны,Мобильные телефоны,None,None,15990.0,36990.0,"{""143"": ""18691"", ""2822"": ""20018"", ""110064"": ""1...","{""143"": ""18691"", ""2822"": ""20018"", ""110064"": ""1...",0
1879552,fffffa8efee9def19a22ffcff0208029418

In [11]:
from functools import cache

In [108]:
# @cache
def tokenize_(sent):
    sent = tokenize(sent)
    return [word.text for word in sent if word.text not in stop and word.text not in punkt]

@cache
def normalize(word):
    try:
        return morph.normal_forms(word)[0]
    except:
        return ''

def lemmatize(sent):
    return ' '.join([normalize(word) for word in sent])

# @cache
def preprocess_sent(sent):
    return lemmatize(tokenize_(sent))

def prepare(row):
    # print(preprocess_sent(row['name_1']))
    ret = ' '.join([
        ' _'.join([''] + preprocess_sent(row['name_1']).split()),
        '~'.join([''] + row['category_level_1_1'].split()) if row['category_level_1_1'] else '',
        '!'.join([''] + row['category_level_2_1'].split()) if row['category_level_2_1'] else '',
        '@'.join([''] + row['category_level_3_1'].split()) if row['category_level_3_1'] else '',
        '#'.join([''] + row['category_level_4_1'].split()) if row['category_level_4_1'] else '',
        preprocess_sent(row['description_1'][:DESC_N].replace('\n', ' ')),
        ' _'.join([''] + preprocess_sent(row['name_2']).upper().split()),
        '~'.join([''] + row['category_level_1_2'].upper().split()) if row['category_level_1_2'] else '',
        '!'.join([''] + row['category_level_2_2'].upper().split()) if row['category_level_2_2'] else '',
        '@'.join([''] + row['category_level_3_2'].upper().split()) if row['category_level_3_2'] else '',
        '#'.join([''] + row['category_level_4_2'].upper().split()) if row['category_level_4_2'] else '',
        preprocess_sent(row['description_2'][:DESC_N].replace('\n', ' ')).upper(),
    ])
    return ret

In [15]:
train_df.iloc[:100].apply(prepare, axis=1)[0]

' _угловой _полка _доить _икона 35 x 35 x 50 забрать сегодня 12.00  _ПОДСВЕЧНИК _ДАВИТЬ _5 ПОДСВЕЧНИК ДАВИТЬ НОМЕР 5 ВЫСОТА 22-23 СМОТРЕТЬ БЕЗ ДОСТАВКА САМОВЫВОЗ'

In [16]:
tqdm.pandas()

In [17]:
# train_df.iloc[9619]

In [18]:
text = train_df.progress_apply(prepare, axis=1)

  5%|███▍                                                                     | 88067/1879555 [02:51<58:12, 513.00it/s]


KeyboardInterrupt: 

In [ ]:
with open('titles_preprocessed_fasttext.pkl', 'rb') as f:
    text = pickle.load(f)

In [24]:
display(text)
display(target)

[' _угловой _полка _доить _икона ~Для~дома~и~дачи !Мебель!и!интерьер @Другое  35 x 35 x 50 забрать сегодня 12.00  _ПОДСВЕЧНИК _ДАВИТЬ _5 ~ДЛЯ~ДОМА~И~ДАЧИ !МЕБЕЛЬ!И!ИНТЕРЬЕР @ПРЕДМЕТЫ@ИНТЕРЬЕРА,@ИСКУССТВО #АРОМАТЫ#ДЛЯ#ДОМА#И#СВЕЧИ ПОДСВЕЧНИК ДАВИТЬ НОМЕР 5 ВЫСОТА 22-23 СМОТРЕТЬ БЕЗ ДОСТАВКА САМОВЫВОЗ',
 ' _угловой _полка _доить _икона ~Для~дома~и~дачи !Мебель!и!интерьер @Другое  35 x 35 x 50 забрать сегодня 12.00  _ТУМБА _ТЕЛЕВИЗОР ~ДЛЯ~ДОМА~И~ДАЧИ !МЕБЕЛЬ!И!ИНТЕРЬЕР @ПОДСТАВКИ@И@ТУМБЫ #ТУМБЫ ТУМБА ХОРОШИЙ СОСТОЯНИЕ',
 ' _угловой _полка _доить _икона ~Для~дома~и~дачи !Мебель!и!интерьер @Другое  35 x 35 x 50 забрать сегодня 12.00  _СВЕЧА ~ДЛЯ~ДОМА~И~ДАЧИ !МЕБЕЛЬ!И!ИНТЕРЬЕР @ПРЕДМЕТЫ@ИНТЕРЬЕРА,@ИСКУССТВО #АРОМАТЫ#ДЛЯ#ДОМА#И#СВЕЧИ РАЗМЕР УПАКОВКА 13 X 18 X 11 ЦЕНА ОДИН СВЕЧА САМОВЫВОЗ',
 ' _наушник _apple _airpods _max _lux ~Электроника !Аудио!и!видео @Наушники  самовывый 3 а hεт только авить доставка пεрεд оформлεh и e м написать сообщεh иε h а вопрос какой адрес куда подъехать ηε отвечат

0          0
1          0
2          0
3          0
4          0
          ..
1879550    0
1879551    0
1879552    0
1879553    0
1879554    0
Name: is_double, Length: 1879555, dtype: int64

In [22]:
del train_df
gc.collect()

3312

In [135]:
def remove_html_tags_and_emoji(text):
    if text is None:
        return None
    clean = re.compile('<.*?>')
    text = re.sub(clean, '', text)
    text = text.replace('\n', ' ')
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"
                               u"\U0001F300-\U0001F5FF"
                               u"\U0001F680-\U0001F6FF"
                               u"\U0001F1E0-\U0001F1FF"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# text = [remove_html_tags_and_emoji(t) for t in tqdm(text)]
# target = target.tolist()

In [162]:
# with open('titles_preprocessed_fasttext.pkl', 'wb') as write_titles:
#     pickle.dump(text, write_titles)

In [ ]:
# if PREPROCESS_DATA:
#     with Pool(16) as p:
#         titles_preprocessed = list(
#             tqdm(
#                 p.imap(
#                     preprocess_sent, 
#                     text
#                 ),
#                 total=len(text)
#             )
#         )
        
#     with open('titles_preprocessed.pkl', 'wb') as write_titles:
#         pickle.dump(titles_preprocessed, write_titles)

In [26]:
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import precision_recall_curve

In [30]:
train_df['text'] = text
# train_df

In [31]:
train_df = train_df.sample(len(train_df), random_state=42)
train_df

,variantid_1,variantid_2,group_id,name_1,name_2,description_1,description_2,category_level_1_1,category_level_1_2,category_level_2_1,...,category_level_3_1,category_level_3_2,category_level_4_1,category_level_4_2,price_1,price_2,characteristic_attributes_mapping_1,characteristic_attributes_mapping_2,is_double,text
1513586,cde98f598e17804a4346f1821f63bd7d8ab5eff0f7e501...,fa310f51c7a25c96e0bd78c8e417fc79b806ee8a591d63...,254861,Пресс гидравлический для склеивания бруса и щита,Пресс для бруса и балок гидравлический,Пресс гидравлический модели SLН - GМ со съемны...,Пресс гидравлический модели SLH - G с подъемны...,Готовый бизнес и оборудование,Готовый бизнес и оборудование,Оборудование для бизнеса,...,Промышленное,Промышленное,Станки,Станки,570000.0,760000.0,"{""181"": ""5119"", ""110064"": ""92826"", ""110408"": ""...","{""181"": ""5119"", ""110064"": ""92826"", ""110408"": ""...",0,_пресс _гидравлический _склеивание _брус _щит...
370259,3265ca22147359315fb33596819ae5535d0aa10001bf29...,f8ddc652025e0ba003894dea3cd6156c68fa3cd214323e...,158367,Шкафчик в туалет,Шкаф платяной,Шкафчик в туалет\nМебель для ванной комнаты ил...,Шкаф платяной\nШкафы купе на заказ по самым лу...,Для дома и дачи,Для дома и дачи,Мебель и интерьер,...,"Шкафы, комоды и стеллажи","Шкафы, комоды и стеллажи",Шкафы и буфеты,Шкафы и буфеты,12440.0,12280.0,"{""45"": ""148"", ""2832"": ""20049"", ""110064"": ""1073...","{""45"": ""148"", ""2832"": ""20049"", ""110064"": ""1122...",1,_шкафчик _туалет ~Для~дома~и~дачи !Мебель!и!и...
1160219,9dcb811baf3861131f58749c5e3c837797462c2cc70efe...,d3750b1d886a5b0792a6c001e41046648c3e6045d69ad6...,150168,Блокхаус. Имитация бревна Сосна. От производителя,Блокхаус. Имитация бревна Сосна. Пиломатериалы,Представленная позиция: Блок хаус Сосна сорт А...,Цена указана при покупке от 150000 рублей или ...,Для дома и дачи,Для дома и дачи,Ремонт и строительство,...,Стройматериалы,Стройматериалы,Пиломатериалы,Пиломатериалы,1140.0,1035.0,"{""44"": ""144"", ""2835"": ""20058"", ""110064"": ""1830...","{""44"": ""144"", ""2835"": ""20058"", ""110064"": ""1830...",0,_блокхаус _имитация _бревно _сосна _от _произ...
1325116,b45b6cf0013c9549d2cd1539f976037e178fb35ec71c99...,7cf6df37880e57b23741f1497c4ee8d13d5ff84699ca59...,189676,Grundfos sba3-45A Колодезный насос автомат,Meibes MeiFlow TOP S UC DN25 Насосная группа б...,"Европейские насосы в наличии на складе: Wilo, ...","Европейские насосы в наличии на складе: DAB,Wi...",Для дома и дачи,Для дома и дачи,Ремонт и строительство,...,"Сантехника, водоснабжение и сауна","Сантехника, водоснабжение и сауна",Водоснабжение и канализация,Водоснабжение и канализация,54000.0,17500.0,"{""44"": ""143"", ""2835"": ""20056"", ""110064"": ""1121...","{""44"": ""143"", ""2835"": ""20056"", ""110064"": ""1121...",0,_grundfos _sba _3-45 _a _колодезный _насос _а...
802133,6cff1322fb5e048aac15261e453c32207916621707cc55...,2fe08e971231a8a193ab27744bec8483cfee168c79e488...,190639,Фикус микропарка,Комнатные растения,Фикус микропарка 1000 рублей.\nГоршок дубок го...,1 хоя\nПо 150 рублей\n\n<ADDRESS-#0>. Бывает р...,Для дома и дачи,Для дома и дачи,Растения,...,None,None,None,None,-1.0,150.0,"{""110064"": ""0"", ""2836"": ""20059""}","{""2836"": ""20059"", ""110064"": ""23947741""}",0,_фикус _микропарк ~Для~дома~и~дачи !Растения ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,236d238fe35c402fe4950a2b681944c69145889fac149d...,84afd8105c3b96eb25d21f55660ec061ef28bf91f84374...,161683,Платье 42-44,Вечернее платье размер 44,"Продам платье в хорошем состоянии, можно на ко...",Красивое элегантное платье с фактурным бантом ...,Личные вещи,Личные вещи,"Одежда, обувь, аксессуары",...,Женская одежда,Женская одежда,Платья,Платья,300.0,300.0,"{""83"": ""3235956"", ""175"": ""747"", ""2826"": ""20029...","{""83"": ""3235956"", ""175"": ""747"", ""2826"": ""20029...",0,"_платье _42-44 ~Личные~вещи !Одежда,!обувь,!а..."
1414414,c06e41bcada1f39d0730dd1bc79dcf281f00cc24690df8...,fd83861

In [32]:
text = train_df['text']
target = train_df['is_double']
groups = train_df['group_id']

In [57]:
import numpy as np

gkf = StratifiedGroupKFold(n_splits=5)
oof_preds = np.zeros(len(text))
# oof_preds = np.zeros(10000)
_it = tqdm(enumerate(gkf.split(text, target, groups)), total=5)
tr_idxxxx = []
val_idxxxx = []
# for fold, (train_idx, val_idx) in enumerate(gkf.split(text[:10000], target[:10000], groups[:10000])):
for fold, (train_idx, val_idx) in _it:
    text_train = [text[i] for i in train_idx]
    y_train = [target[i] for i in train_idx]
    text_val = [text[i] for i in val_idx]
    y_val = [target[i] for i in val_idx]
    
    tr_idxxxx.append(train_idx)
    val_idxxxx.append(val_idx)
    
    _it.set_description('writing train to file')
    with open(f'train_data_fold{fold}.txt', 'w+', encoding='utf-8') as tr:
        for idx in range(len(text_train)):
            tr.write('__label__' + str(y_train[idx]) + ' ' + text_train[idx] + '\n')

    _it.set_description('writing val to file')
    with open(f'val_data_fold{fold}.txt', 'w+', encoding='utf-8') as valf:
        for idx in range(len(text_val)):
            valf.write('__label__' + str(y_val[idx]) + ' ' + text_val[idx] + '\n')
        # print('__label__' + str(y_val[idx]) + ' ' + text_val[idx])
    
    _it.set_description('training')
    # ft_model = fasttext.train_supervised(
    #     input=f'train_data_fold{fold}.txt',
    #     dim=10,
    # )
    _it.set_description('saving model')
    # ft_model.save_model(f'fast_avito_fold{fold}.model')

    _it.set_description('predicting')
    # val_preds = []
    # for text_ in text_val:
    #     pred = ft_model.predict(text_)
    #     # if pred[0][0][-1] == '0':
    #     #     print(pred)
    #     #     print(pred[1][0] if pred[0][0][-1] == '1' else 1-pred[1][0])
    #     val_preds.append(pred[1][0] if pred[0][0][-1] == '1' else 1-pred[1][0])
    
    # oof_preds[val_idx] = val_preds
_it.close()

predicting: 100%|████████████████████████████████████████████████████████████████████████| 5/5 [03:57<00:00, 47.57s/it]


In [104]:
oof_preds_bad_format = []
for nfold in range(5):
    m = fasttext.train_supervised(f'train_data_fold{nfold}.txt', verbose=10000, dim=300)
    print('fold', nfold, 'train score', *m.test(f'train_data_fold{nfold}.txt'))
    print('fold', nfold, 'val score', *m.test(f'val_data_fold{nfold}.txt'))
    arr = []
    with open(f'val_data_fold{nfold}.txt', 'r', encoding='utf-8') as f:
        for line in tqdm(f.readlines()): 
            p = m.predict(line[11:].strip())
            arr.append(p[1][0] if p[0][0][-1] == '1' else 1 - p[1][0])
    y_val = [target[i] for i in val_idxxxx[nfold]]
    precision, recall, thresholds = precision_recall_curve(y_val, arr)
    oof_pr_auc = auc(recall, precision)
    oof_preds_bad_format.append(arr[:])
    print(f'for {nfold}\'th fold {oof_pr_auc=}')
    m.save_model(f'fast_avito_fold{nfold}_d300.model')

fold 0 train score 1503646 0.960291185558303 0.960291185558303
fold 0 val score 375909 0.9574923718240319 0.9574923718240319


100%|███████████████████████████████████████████████████████████████████████| 375909/375909 [00:30<00:00, 12428.29it/s]


for 0'th fold oof_pr_auc=0.5669954214344956
fold 1 train score 1503634 0.9602383292742782 0.9602383292742782
fold 1 val score 375921 0.957408604467428 0.957408604467428


100%|███████████████████████████████████████████████████████████████████████| 375921/375921 [00:33<00:00, 11283.46it/s]


for 1'th fold oof_pr_auc=0.5789653930607889
fold 2 train score 1503646 0.9602286708440684 0.9602286708440684
fold 2 val score 375909 0.9572157091210904 0.9572157091210904


...

for 2'th fold oof_pr_auc=0.5688473552918799
fold 3 train score 1503646 0.9603231079655716 0.9603231079655716
fold 3 val score 375909 0.9572955156700159 0.9572955156700159


100%|███████████████████████████████████████████████████████████████████████| 375909/375909 [00:30<00:00, 12281.82it/s]


for 3'th fold oof_pr_auc=0.575189470474549
fold 4 train score 1503648 0.960299218966141 0.960299218966141
fold 4 val score 375907 0.9570718289364125 0.9570718289364125


100%|███████████████████████████████████████████████████████████████████████| 375907/375907 [00:30<00:00, 12297.11it/s]


for 4'th fold oof_pr_auc=0.5717739465257025


In [142]:
with open('oof_preds_bad_format.pkl', 'wb') as w:
    pickle.dump(oof_preds_bad_format, w)

In [83]:
import matplotlib.pyplot as plt
# plt.hist(oof_preds, bins=100)

In [97]:
from sklearn.metrics import auc, roc_curve, RocCurveDisplay, f1_score

In [110]:
# precision, recall, thresholds = precision_recall_curve(target, oof_preds)
# precision, recall, thresholds = precision_recall_curve(target[:10000], oof_preds[:10000])
# oof_pr_auc = auc(recall, precision)
# print(f'{oof_pr_auc=}')

In [106]:
# fpr, tpf, thr = roc_curve(y_val, arr)
# print(auc(fpr, tpf))
# RocCurveDisplay(fpr=fpr, tpr=tpf).plot()

In [107]:
test_df = pd.read_parquet('test_zalupa.parquet')
test_df

,variantid_1,variantid_2,name_1,name_2,description_1,description_2,category_level_1_1,category_level_1_2,category_level_2_1,category_level_2_2,category_level_3_1,category_level_3_2,category_level_4_1,category_level_4_2,price_1,price_2,characteristic_attributes_mapping_1,characteristic_attributes_mapping_2
0,00001e4f563a4dc91c63663a8b64068b82b5d046cc7b24...,883cb78f30a3cb90b26c674c4477464c11ac4fca8bf713...,Коврики для мышки Meow Gaming Gear (MGG),IYX MU68 PRO,ПЕРЕД ОФОРМЛЕНИЕМ АВИТО ДОСТАВКИ ПИШИТЕ В ЛС! ...,"ПЕРЕД ОФОРМЛЕНИЕМ АВИТО ДОСТАВКИ ПИШИТЕ, ПОЖАЛ...",Электроника,Электроника,Товары для компьютера,Товары для компьютера,Аксессуары,Клавиатуры и мыши,None,None,3890.0,10490.0,"{""483"": ""6666"", ""2823"": ""20021"", ""110064"": ""10...","{""483"": ""5022"", ""2823"": ""20021"", ""110064"": ""10..."
1,00001e4f563a4dc91c63663a8b64068b82b5d046cc7b24...,afd827a7e59d8e855f0079b18d78048495757e2a2a4351...,Коврики для мышки Meow Gaming Gear (MGG),Веб-камера Logitech StreamCam,ПЕРЕД ОФОРМЛЕНИЕМ АВИТО ДОСТАВКИ ПИШИТЕ В ЛС! ...,"ПЕРЕД ОФОРМЛЕНИЕМ АВИТО ДОСТАВКИ ПИШИТЕ, ПОЖАЛ...",Электроника,Электроника,Товары для компьютера,Товары для компьютера,Аксессуары,Веб-камеры,None,None,3890.0,9650.0,"{""483"": ""6666"", ""2823"": ""20021"", ""110064"": ""10...","{""483"": ""5020"", ""2823"": ""20021"", ""110064"": ""10..."
2,00001e4f563a4dc91c63663a8b64068b82b5d046cc7b24...,d4969ce50032645ba4dfea0d4aeeb0316df579788caa22...,Коврики для мышки Meow Gaming Gear (MGG),Коврики для мышки Vaxee PA в ассортименте,ПЕРЕД ОФОРМЛЕНИЕМ АВИТО ДОСТАВКИ ПИШИТЕ В ЛС! ...,"❗️Не оформляйте заказ, не договорившись с нами...",Электроника,Электроника,Товары для компьютера,Товары для компьютера,Аксессуары,Аксессуары,None,None,3890.0,5990.0,"{""483"": ""6666"", ""2823"": ""20021"", ""110064"": ""10...","{""483"": ""6666"", ""2823"": ""20021"", ""110064"": ""10..."
3,0000649b9fb42fec39328949bc74877e98f6687714136d...,ba6045b39544dd904b7be112d0802ee1fe15d279df137e...,Клюшка хоккейная bauer 1x se vapor,Клюшка хоккейная bauer xlite,Продаю хоккейную клюшку Bauer Vapor 1X SE для ...,Продаю хоккейную клюшку Bauer Vapor 1X Lite дл...,Хобби и отдых,Хобби и отдых,Спорт и отдых,Спорт и отдых,Зимний спорт,Зимний спорт,Хоккей,Хоккей,6000.0,5000.0,"{""165"": ""685"", ""2843"": ""20122"", ""110064"": ""147...","{""165"": ""685"", ""2843"": ""20122"", ""110064"": ""147..."
4,00010817f43942cd236fc300f0d8d497127cf48bd65245...,04b0e2595bf4fe466418d4158f8acf724549825519676a...,Продвижение экспертов (специалистов ) в соц сетях,Продвижение бизнеса на картах (геомаркетинг),🔎 Деньги на любимом деле ! \n\nКейсы:\n\nHишa:...,"⭐️⭐️⭐️⭐️⭐️ геомаркетинг \nЯндекс, 2гиc\n\nХоти...",Услуги,Услуги,Предложение услуг,Предложение услуг,"IT, дизайн, маркетинг","IT, дизайн, маркетинг","Реклама, PR, маркетинг","Реклама, PR, маркетинг",1000.0,5000.0,"{""716"": ""10195"", ""1371"": ""3262743"", ""110064"": ...","{""716"": ""10195"", ""1371"": ""3262743"", ""110064"": ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,ffff336a8447d1c9f24c47fc85af5edd26671d68171523...,8ee26e594c6ab4ed3f368eee38b14ae7a8fd4712ecfe5a...,Чехол на iPhone 13,Чехол Samsung S21fe 5g,Чехол из кевлара на айфон iphone 13 \nMageSaf...,Чехол от Самсунг S21fe 5g. Состояние нового. \...,Электроника,Электроника,Телефоны,Телефоны,Аксессуары,Аксессуары,Чехлы и плёнки,Чехлы и плёнки,1599.0,25.0,"{""143"": ""4987"", ""480"": ""4993"", ""2822"": ""20018""...","{""143"": ""4987"", ""480"": ""4993"", ""2822"": ""20018""..."
499996,ffff6b7f2e635225076944eb333e422d4a19d87d6c5b41...,42814661b581f992113b20e73b7386f5b5bc7be4f44105...,Formula Ice FR 235/45 R18 98T,Formula Ice 235/55 R18 104T,Шины зимние нешипованные Formula Ice FR 235/45...,Шины зимние шипованные Formula Ice 235/55 R18 ...,Транспорт,Транспорт,Запчасти и аксессуары,Запчасти и аксессуары,"Шины, диски и колёса","Шины, диски и колёса",Легковые шины,Легковые шины,12150.0,11600.0,"{""5"": ""19"", ""709"": ""10048"", ""731"": ""10323"", ""7...","{""5"": ""19"",

In [109]:
test_texts = test_df.progress_apply(prepare, axis=1)
test_texts

100%|█████████████████████████████████████████████████████████████████████████| 500000/500000 [12:38<00:00, 659.39it/s]


0          _коврик _мышка _meow _gaming _gear _mgg ~Элек...
1          _коврик _мышка _meow _gaming _gear _mgg ~Элек...
2          _коврик _мышка _meow _gaming _gear _mgg ~Элек...
3          _клюшка _хоккейный _bauer _1 _x _se _vapor ~Х...
4          _продвижение _эксперт _специалист _соц _сеть ...
                                ...                        
499995     _чехол _iphone _13 ~Электроника !Телефоны @Ак...
499996     _formula _ice _fr _235/45 _r _18 _98 _t ~Тран...
499997     _куртка _собака ~Животные !Товары!для!животны...
499998     _барабан _тормозной _toyota _corolla ~Транспо...
499999     _новый _дождевик _собака _мелкий _порода ~Жив...
Length: 500000, dtype: object

In [137]:
test_texts_no_emj = test_texts.progress_apply(remove_html_tags_and_emoji)

100%|███████████████████████████████████████████████████████████████████████| 500000/500000 [00:10<00:00, 46554.92it/s]


In [114]:
models = [fasttext.load_model(f'fast_avito_fold{i}_d300.model') for i in range(5)]

In [127]:
def get_avg_prediction(text, models):
    preds = []
    for model in models:
        p = model.predict(text)
        preds.append(p[1][0] if p[0][0][-1] == '1' else 1 - p[1][0])
    return np.mean(preds)

In [138]:
get_avg_prediction(test_texts_no_emj[0], models)

0.027574241161346436

In [ ]:
# models = [fasttext.load_model(f'fast_avito_fold{fold}.model') for fold in range(5)]

In [140]:
test_preds = [get_avg_prediction(text, models) for text in tqdm(test_texts_no_emj)]


100%|████████████████████████████████████████████████████████████████████████| 500000/500000 [03:18<00:00, 2514.08it/s]


In [141]:
with open('test_preds.pkl', 'wb') as w:
    pickle.dump(test_preds, w)